## Estrutura de armazenamento dos dados

```
    semester_id
             |--- class_id
             |        |--- assessment_id
             |        |      |--- {feature1 : feature_value}
             |        |      |--- {feature2 : feature_value}
             |        |      |--- .......
             |        |      |--- {featureN : feature_value}
```

## Extraindo os dados dos arquivos

### Importando bibliotecas

In [ ]:
import tarfile
import os
import re
import pymysql

from pprint import pprint

### Coletando os semestres

In [ ]:
PATH = '../dataset/workbench'
dataset = {}
for semester_id in os.listdir(PATH):
    if re.search(r'\d{4}-\d', semester_id):
        dataset[semester_id] = {}

pprint(dataset)

### Coletando as turmas

In [ ]:
PATH = '../dataset/workbench/{semester_id}'

for semester_id in dataset.keys():
    path = PATH.replace('{semester_id}', semester_id)
    for class_id in os.listdir(path):
        if re.match(r'\d+', class_id):
            dataset[semester_id][class_id] = {}

pprint(dataset)

### Coletando os id's das avaliações

In [ ]:
PATH = '../dataset/workbench/{semester_id}/{class_id}/assessments/'

for semester_id in dataset.keys():
    # print(semester_id)
    for class_id in dataset[semester_id].keys():
        path = PATH.replace('{semester_id}', semester_id)
        path = path.replace('{class_id}', class_id)
        # print(' ', class_id)
        for assessment_id in os.listdir(path):
            #print('  ', assessment_id)
            assessment_id = re.sub(r'.data','', assessment_id)
            dataset[semester_id][class_id][assessment_id] = {}
        # print(f' class_id={class_id}, path={path}')

pprint(dataset)

### Extraindo as características e os valores

In [ ]:
# create assessment features and values
ROOT = '../dataset/workbench/{semester_id}/{class_id}/assessments/'

def extract_feature_value(assessment_file):
    features = []
    for line in assessment_file:
        feature = re.search(r'(\-{4}\s)([a-z\s\_]+)(\:)', line)
        value = re.search(r'([a-z]\:\s)(.+)', line)
        if feature and value:
            feature = feature.group(2)
            feature = re.sub(r'\s', '_', feature)
            features.append(feature)
            value = value.group(2)
            values.append(value)
    return features, values

for semester_id in dataset.keys():
    for class_id in dataset[semester_id].keys():
        path = ROOT.replace('{semester_id}', semester_id)
        path = path.replace('{class_id}', class_id)
        for assessment_id in dataset[semester_id][class_id].keys():
            assessment_file_name = assessment_id+'.data'
            fullpath = os.path.join(path, assessment_file_name)
            features, values = [], []
            with open(fullpath, 'r') as assessment_file:
                features, values = extract_feature_value(assessment_file)
            for feature, value in zip(features, values): 
                #print(f'semester_id={semester_id}, class_id={class_id}, assessment_id={assessment_id}, feature={feature}, value={value}')
                dataset[semester_id][class_id][assessment_id][feature] = value

pprint(dataset)

## Inserido dados no banco de dados

### Estabelecendo conxeão

In [ ]:
def connection_database():
    connection = pymysql.connect(host='localhost',
                             user='root',
                             password='open',
                             db='dataset-workbench',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
    return connection

### Gravando no banco

In [ ]:
conn = connection_database()
cursor = conn.cursor()

for semester_id in dataset.keys():
    for class_id in dataset[semester_id].keys():
        for assessment_id in dataset[semester_id][class_id].keys():
            features = f"assessment_id, semester_id, "
            values = f"'{semester_id}#{class_id}#{assessment_id}', '{semester_id}', "
            for feature in dataset[semester_id][class_id][assessment_id].keys():
                feature_value = dataset[semester_id][class_id][assessment_id][feature] 
                return_search = re.search(r'\d+\:\d+', feature_value)
                if return_search:
                    hour = return_search.group(0)
                    new_hour = hour+':00'
                    feature_value = feature_value.replace(hour, new_hour)

                features += f"{feature}, "
                if feature in ['assessment_title', 'class_name',
                                'start', 'end', 'language', 'codemirror_mode',
                                'type']:
                    values += f"'{feature_value}', "
                else:
                    values += f"{feature_value}, "
            insert_assessment = f"INSERT INTO assessment ({features[:-2]}) VALUES ({values[:-2]})"
            print(insert_assessment)
            cursor.execute(insert_assessment)

conn.commit()
conn.close()